In [1]:
import numpy as np
from alma import *
#from almapyc import *

In [2]:
alma_params = tomlload('params.toml')

# Check none for parallel
if type(alma_params['parallel']) is not bool:
    alma_params['parallel'] = None

alma_params

{'filename': '/mnt/c/Users/MLL591/OneDrive/Research/PyALMA3/EuropaProfile_Seawater_35.2ppt_Tb268.305K.txt',
 'rheology_structure': ['elastic', 'newton', 'maxwell'],
 'layer_radius': False,
 'layer_radius_index': False,
 'num_digits': 128,
 'gorder': 8,
 'tau': 0,
 'mode': 'tidal',
 'dmin': 1,
 'dmax': 10,
 'dstep': 1,
 'time': 'log',
 'tpoints': 20,
 'timestart': -3,
 'timeend': 2,
 'function': 'step',
 'function_length': 1.0,
 'verbose': True,
 'type': 'real',
 'parallel': None}

In [3]:
# Read PP model
model = read_model_pp(abspath(alma_params['filename']))

In [4]:
rheology, params = infer_rheology_pp(model,
                                         structure=alma_params['rheology_structure'],
                                         layer_radius=alma_params['layer_radius'],
                                         layer_radius_index=alma_params['layer_radius_index'])

  >> Verifying validity of structure rheologies
  >> Could not find pre-defined layers from radius.
     Infering layers from density profile.


In [5]:
model_params = build_model(model['model'][:, model['columns'].index('r')],
                               model['model'][:, model['columns'].index('rho')],
                               model['mu'],
                               model['vis'],
                               rheology,
                               params,
                               ndigits = alma_params['num_digits'],
                               verbose = alma_params['verbose'])

> Initializing
  >> Setting precision: 128
> Building model
  >> Parsing Rheology
  >> Computing mass of the planet
  >> Computing gravity at the interface boundaries
  >> Normalizing model parameters


In [6]:
# Harmonic degrees
n = arange(alma_params['dmin'], alma_params['dmax'], alma_params['dstep'])

In [7]:
# Time step
if alma_params['time'] == 'lin':
    t = linspace(alma_params['timestart'], 
                 alma_params['timeend'], 
                 alma_params['tpoints'],
                 endpoint=True)
elif alma_params['time'] == 'log':
    t = logspace(alma_params['timestart'], 
                 alma_params['timeend'], 
                 alma_params['tpoints'],
                 endpoint=True)
elif alma_params['time'] == 'geom':
    t = geomspace(alma_params['timestart'], 
                 alma_params['timeend'], 
                 alma_params['tpoints'],
                 endpoint=True)
else:
    print(f'ERROR: Unknown scale for time: {alma_params["time"]}.')
    print('Allowed values are: "log", "geom", "lin".')
    raise ValueError()

In [ ]:
# Compute love numbers
h, l, k = love_numbers(n, t, 
                       alma_params['mode'], 
                       alma_params['function'],
                       alma_params['tau'],
                       model_params,
                       alma_params['type'],
                       alma_params['gorder'],
                       verbose = alma_params['verbose'],
                       parallel = alma_params['parallel'])

> Computing Salzer weights
> Computing Love Numbers - Parallel


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 14 concurrent workers.
